In [1]:
import matplotlib.pyplot as plt
import scipy.io
import seaborn as sns
import numpy as np
import pandas as pd
import networkx as nx
import scipy.io as sio
import networkx as nx
import seaborn as sns
from scipy.signal import coherence, hilbert
import scipy.io

In [5]:
import mne
raw = mne.io.read_raw_edf(r"C:/Users/rayom/OneDrive/Documentos/GitHub/Neurociencias-2026-1/S03_datasets/EEG_practica", preload=True)

NotImplementedError: Only EDF files are supported, got .